In [58]:
print("Hello|")

Hello|


In [59]:
pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.


In [60]:
pip install -U langchain pinecone-client ctransformers

Note: you may need to restart the kernel to use updated packages.


In [61]:
pip install pymupdf

Note: you may need to restart the kernel to use updated packages.


In [62]:
pip install -U langchain-huggingface


Note: you may need to restart the kernel to use updated packages.


In [63]:
pip install -U pinecone-client

Note: you may need to restart the kernel to use updated packages.


In [64]:
from tqdm.autonotebook import tqdm

In [65]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain.document_loaders import PyMuPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import ctransformers

In [66]:
PINECONE_API_KEY = "fa012826-08d2-41a5-a61b-9c1ee273defe"

In [67]:
#Extract data from PDF files
def load_pdf(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyMuPDFLoader)
    documents = loader.load()

    return documents



In [68]:
extracted_data = load_pdf("data/")

d:\Code\Medical chatbot\Medical-Chatbot-using-llama\medical\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:299: UserWarning: Warning: Empty content on page 0 of document data\Gale Encyclopedia of Medicine. Vol. 1. 2nd Edition ( PDFDrive ).pdf
  warnings.warn(


In [69]:
# Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [70]:
text_chunks = text_split(extracted_data)
print("length of my chunks: ", len(text_chunks))

length of my chunks:  5779


In [71]:
#download embeddings model
def download_huggingface_model():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/paraphrase-MiniLM-L6-v2")
    return embeddings

In [72]:
embeddings = download_huggingface_model()

d:\Code\Medical chatbot\Medical-Chatbot-using-llama\medical\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [73]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='sentence-transformers/paraphrase-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [74]:
query_result = embeddings.embed_query("Hello, how are you?")
print("length of my query result: ", len(query_result))

length of my query result:  384


In [75]:
query_result

[0.5421559810638428,
 0.18908195197582245,
 0.8348346948623657,
 0.019833765923976898,
 -0.8182635307312012,
 -0.5520739555358887,
 0.5031420588493347,
 0.11975957453250885,
 -0.2559368312358856,
 0.2397879958152771,
 0.06577672064304352,
 -0.4289025068283081,
 -0.24317021667957306,
 -0.14757925271987915,
 0.21157336235046387,
 0.27461254596710205,
 0.22511503100395203,
 -0.4708331525325775,
 -0.8574920296669006,
 -0.02177487686276436,
 -0.44853755831718445,
 -0.03448502719402313,
 -0.11906781792640686,
 -0.015861667692661285,
 0.34434258937835693,
 -0.016430480405688286,
 0.012980563566088676,
 0.6993362307548523,
 0.033979810774326324,
 0.05523053556680679,
 -0.1583712249994278,
 -0.16747117042541504,
 -0.10531020164489746,
 0.15381643176078796,
 0.1834600418806076,
 0.3758128881454468,
 0.10488469153642654,
 0.10315363109111786,
 -0.18420681357383728,
 0.17847377061843872,
 0.00675179623067379,
 -0.3362417221069336,
 -0.04654860123991966,
 -0.07281830161809921,
 0.3359984755516052,


# Pinecone

In [76]:
pip install "pinecone-client[grpc]"

Note: you may need to restart the kernel to use updated packages.


In [93]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

# Initialize a client
pc = Pinecone(api_key='fa012826-08d2-41a5-a61b-9c1ee273defe')

# Define index name
index_name = "medical"

# Extract just the names of the indexes
existing_index= []
for index in pc.list_indexes():
    existing_index.append(index['name'])

# Print the list of index names for debugging
print("Existing index names:", existing_index)

# Check if the index already exists
if index_name in existing_index:
    print("Index is already created.")
else:
    # Try to create the serverless index
    try:
        pc.create_index(
            name=index_name,
            dimension=384,
            metric="cosine",
            spec=ServerlessSpec(
                cloud="aws",
                region="us-east-1"
            ),
            deletion_protection="disabled"
        )
        print("Index created.")
    except Exception as e:
        print(f"Error creating index: {e}")


Existing index names: ['medical']
Index is already created.


In [94]:
pc.describe_index(index_name)

{'deletion_protection': 'disabled',
 'dimension': 384,
 'host': 'medical-667fvl4.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'medical',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}

In [95]:
index= pc.Index(index_name)
index.fetch(["id-1", "id-2"])

{'namespace': '', 'usage': {'read_units': 0}, 'vectors': {}}

In [96]:
import os
index = pc.Index(host='medical-667fvl4.svc.aped-4627-b74a.pinecone.io')

In [97]:
# from langchain.vectorstores import Pinecone as PC
# Create a Pinecone index object
index = pc.Index(index_name)

# Prepare the texts and embeddings for storage
texts = [t.page_content for t in text_chunks]
docsearch = index.fetch(texts, embeddings, index_name=index_name)

TypeError: bad argument type for built-in operation

In [100]:
# Initialize Pinecone Index
index = pc.Index(index_name)

def batch_upsert(index, vectors, batch_size=1000):
    for i in range(0, len(vectors), batch_size):
        batch = vectors[i:i + batch_size]
        upsert_response = index.upsert(vectors=batch)
        print(f"Upsert response for batch {i//batch_size + 1}: {upsert_response}")

# Prepare the embeddings
embeddings_to_upsert = [
    (f"vec{i}", embeddings.embed_documents([t.page_content])[0], {"content": t.page_content}) 
    for i, t in enumerate(text_chunks)
]

# Call the batch upsert function
batch_upsert(index, embeddings_to_upsert, batch_size=1000)


Upsert response for batch 1: upserted_count: 1000

Upsert response for batch 2: upserted_count: 1000

Upsert response for batch 3: upserted_count: 1000

Upsert response for batch 4: upserted_count: 1000

Upsert response for batch 5: upserted_count: 1000

Upsert response for batch 6: upserted_count: 779

